In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19744312
paper_name = 'bivi_delneri_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_excel('raw_data/13059_2009_2243_MOESM2_ESM.xls', skiprows=5)
original_data2 = pd.read_excel('raw_data/13059_2009_2243_MOESM3_ESM.xls', skiprows=4)

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 511 x 12
Original data dimensions: 134 x 9


In [7]:
original_data1.head()

,Unnamed: 0,ORF,Growth Rate Difference,FDR-Value,Unnamed: 4,ORF.1,Growth Rate Difference.1,FDR-Value.1,Unnamed: 8,ORF.2,Growth Rate Difference.2,FDR-Value.2
0,NaN,YNR024W,NaN,NaN,NaN,YNR024W,NaN,NaN,NaN,YNR024W,NaN,NaN
1,NaN,YLR197W,NaN,NaN,NaN,YBL052C,NaN,NaN,NaN,YBL052C,NaN,NaN
2,NaN,YNR038W,NaN,NaN,NaN,YPL228W,NaN,NaN,NaN,YLR197W,NaN,NaN
3,NaN,YDR398W,NaN,NaN,NaN,YLR197W,NaN,NaN,NaN,YNR038W,NaN,NaN
4,NaN,YOR366W,NaN,NaN,NaN,YNR038W,NaN,NaN,NaN,YDR398W,NaN,NaN


In [8]:
all_orfs = pd.concat([original_data1['ORF'], original_data1['ORF.1'], original_data1['ORF.2'], original_data2['ORF'], original_data2['ORF.1']], axis=0)

In [9]:
all_orfs = all_orfs.unique()
all_orfs.shape

(705,)

In [10]:
original_data = pd.DataFrame(index=all_orfs, columns=['RIS','ALE','IBA'], data=np.nan)

## First, prep strains that passed QC

In [11]:
original_data4 = pd.read_excel('raw_data/13059_2009_2243_MOESM4_ESM.xls', skiprows=1)
original_data5 = pd.read_excel('raw_data/13059_2009_2243_MOESM5_ESM.xls', skiprows=1)

In [12]:
original_data5.head()

,IBA,ALE,ALE and IBA
0,YPL173W,YPL014W,YJL111W
1,YPL093W,YOR023C,YLL004W
2,YOR165W,YPL029W,YNL103W
3,YKL107W,YPR188C,YJL051W
4,YHR066W,YPL174C,YLL025W


In [13]:
iba_hi = pd.concat([original_data4['Common'], original_data4['IBA unique'], original_data4['RIS and IBA'], original_data4['IBA and ALE']], axis=0)
iba_hi = list(iba_hi.unique())
iba_hi = [x for x in iba_hi if not isinstance(x, float)]
len(iba_hi)

449

In [14]:
ale_hi = pd.concat([original_data4['Common'], original_data4['ALE unique'], original_data4['RIS and ALE'], original_data4['IBA and ALE']], axis=0)
ale_hi = list(ale_hi.unique())
ale_hi = [x for x in ale_hi if not isinstance(x, float)]
len(ale_hi)

312

In [15]:
ris_hi = pd.concat([original_data4['Common'], original_data4['RIS Unique'], original_data4['RIS and IBA'], original_data4['RIS and ALE']], axis=0)
ris_hi = list(ris_hi.unique())
ris_hi = [x for x in ris_hi if not isinstance(x, float)]
len(ris_hi)

253

In [16]:
iba_hp = pd.concat([original_data5['IBA'], original_data5['ALE and IBA']], axis=0)
iba_hp = list(iba_hp.unique())
iba_hp = [x for x in iba_hp if not isinstance(x, float)]
len(iba_hp)

124

In [17]:
ale_hp = pd.concat([original_data5['ALE'], original_data5['ALE and IBA']], axis=0)
ale_hp = list(ale_hp.unique())
ale_hp = [x for x in ale_hp if not isinstance(x, float)]
len(ale_hp)

15

## Next, load the QOFH data

In [18]:
t = original_data1[['ORF','Growth Rate Difference']]
t = t.loc[t['ORF'].notnull()]
t.loc[t['Growth Rate Difference'].isnull(),'Growth Rate Difference'] = -2  # Quick disappearing (QD) strains, i.e. present at t=0 but absent at all other timepoints), and therefore severely haploinsufficient.
t = t.loc[t['ORF'].isin(ris_hi)]
original_data.loc[t['ORF'].values,'RIS'] = t['Growth Rate Difference'].values

In [19]:
t = original_data1[['ORF.1','Growth Rate Difference.1']]
t = t.loc[t['ORF.1'].notnull()]
t.loc[t['Growth Rate Difference.1'].isnull(),'Growth Rate Difference.1'] = -2  # Quick disappearing (QD) strains, i.e. present at t=0 but absent at all other timepoints), and therefore severely haploinsufficient.
t = t.loc[t['ORF.1'].isin(ale_hi)]
original_data.loc[t['ORF.1'].values,'ALE'] = t['Growth Rate Difference.1'].values

In [20]:
t = original_data1[['ORF.2','Growth Rate Difference.2']]
t = t.loc[t['ORF.2'].notnull()]
t.loc[t['Growth Rate Difference.2'].isnull(),'Growth Rate Difference.2'] = -2  # Quick disappearing (QD) strains, i.e. present at t=0 but absent at all other timepoints), and therefore severely haploinsufficient.
t = t.loc[t['ORF.2'].isin(iba_hi)]
original_data.loc[t['ORF.2'].values,'IBA'] = t['Growth Rate Difference.2'].values

In [21]:
t = original_data2[['ORF','Growth Rate']]
t = t.loc[t['ORF'].notnull()]
t = t.loc[t['ORF'].isin(ale_hp)]
original_data.loc[t['ORF'].values,'ALE'] = t['Growth Rate'].values

In [22]:
t = original_data2[['ORF.1','Growth Rate.1']]
t = t.loc[t['ORF.1'].notnull()]
t = t.loc[t['ORF.1'].isin(iba_hp)]
original_data.loc[t['ORF.1'].values,'IBA'] = t['Growth Rate.1'].values

In [23]:
original_data.notnull().sum(axis=0)

RIS    248
ALE    327
IBA    573
dtype: int64

In [24]:
print(original_data.shape)
original_data = original_data.loc[original_data.notnull().sum(axis=1)>0]
print(original_data.shape)

(705, 3)
(637, 3)


In [25]:
original_data.head()

,RIS,ALE,IBA
YNR024W,-2.0,-2.0,-2.0
YLR197W,-2.0,-2.0,-2.0
YNR038W,-2.0,-2.0,-2.0
YDR398W,-2.0,-2.0,-2.0
YGR037C,-2.0,-2.0,-2.0


In [26]:
original_data.min(axis=0)

RIS   -2.0
ALE   -2.0
IBA   -2.0
dtype: float64

In [27]:
original_data = original_data.reset_index()
original_data.columns = ['orf','RIS','ALE','IBA']

## Normal processing and filtering

In [28]:
original_data['orf'] = original_data['orf'].astype(str)

In [29]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [30]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [31]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, RIS, ALE, IBA]
Index: []


In [32]:
original_data.set_index('orf', inplace=True)

In [33]:
original_data = original_data.groupby(original_data.index).mean()

In [34]:
original_data.shape

(637, 3)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [22233,22231,22232]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,22233,22231,22232
data_type,value,value,value
orf,,,
YAL005C,NaN,NaN,-0.072294
YAL019W,NaN,NaN,-2.000000
YAL023C,NaN,NaN,-0.139898
YAL036C,-0.100414,NaN,-2.000000
YAL044C,-2.000000,-2.0,-2.000000


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22233,22231,22232
,data_type,value,value,value
gene_id,orf,,,
4,YAL005C,NaN,NaN,-0.072294
17,YAL019W,NaN,NaN,-2.000000
21,YAL023C,NaN,NaN,-0.139898
34,YAL036C,-0.100414,NaN,-2.000000
42,YAL044C,-2.000000,-2.0,-2.000000


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=False, space='het')

In [42]:
data_norm.shape

(5304, 3)

In [48]:
data_norm.head()

,dataset_id,22233,22231,22232
,data_type,valuez,valuez,valuez
gene_id,orf,,,
647,YCR099C,0.0,-0.022557,-0.023053
5955,YPR068C,0.0,-0.022557,-0.023053
74,YAR029W,0.0,-0.022557,-0.023053
3772,YLR083C,0.0,-0.022557,-0.023053
3223,YGR279C,0.0,-0.022557,-0.023053


In [43]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [56]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22233,22231,22232,22233,22231,22232
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
4,YAL005C,NaN,NaN,-0.072294,NaN,NaN,-0.184165
17,YAL019W,NaN,NaN,-2.000000,NaN,NaN,-4.480177
21,YAL023C,NaN,NaN,-0.139898,NaN,NaN,-0.334825
34,YAL036C,-0.100414,NaN,-2.000000,-0.259672,NaN,-4.480177
42,YAL044C,-2.000000,-2.0,-2.000000,-5.172047,-4.626685,-4.480177


# Print out

In [59]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')